Table of Contents
===

<a href="#about">About this notebook</a> <br />
<a href="#setup">Setting things up</a>

Local Experience
1. <a href="#local_preprocessing">Local preprocessing starting from csv files</a>
1. <a href="#local_training">Local training</a>
1. <a href="#local_prediction">Local prediction</a>
1. <a href="#local_batch_prediction">Local batch prediction</a>

<a name="about"></a>
About this notebook
======

This notebook uses the datalab structured data package for building and running a Tensorflow classification problems locally. This notebook uses the classic <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set">Iris flower data set.</a>

In the notebooks that follow, an example of running preprocessing, training, and prediction using the Google Cloud Machine Learning Engine services are given. Note that running the cloud versions of preprocessing, training, and prediction take longer than the local versions. The performance advantage of using the cloud applies to very large data sets, and you don't see it with this sample because the data is small and run time is dominated by setup overhead.

<a name="setup"></a>
Setting things up
=====

In [33]:
import datalab_structured_data as sd

Lets look at the versions of datalab_structured_data and TF we have. Make sure TF and SD are 1.0.0

In [34]:
import tensorflow as tf
from tensorflow.python.lib.io import file_io
import datalab.ml as ml
import datalab
import json
import os
print('tf ' + str(tf.__version__))
print('sd ' + str(sd.__version__))

tf 1.0.0
sd 1.0.0


This notebook will write files during preprocessing, training, and prediction. Please give a root folder you wish to use.

In [35]:
# Edit LOCAL_ROOT if you want to save files to a different location.
# otherwise don't change anything is this cell. If the folder
# already exists, it will be deleted.
LOCAL_ROOT = './iris_notebook_workspace'

# No need to edit anything else in this cell. But if you do, you 
# might need to chagne the global variables in the cloud notebooks.
LOCAL_PREPROCESSING_DIR = os.path.join(LOCAL_ROOT, 'preprocessing')
LOCAL_TRAINING_DIR = os.path.join(LOCAL_ROOT, 'training')
LOCAL_BATCH_PREDICTION_DIR = os.path.join(LOCAL_ROOT, 'batch_prediction')

LOCAL_TRAIN_FILE = os.path.join(LOCAL_ROOT, 'train.csv')
LOCAL_EVAL_FILE = os.path.join(LOCAL_ROOT, 'eval.csv')
LOCAL_PREDICT_FILE = os.path.join(LOCAL_ROOT, 'predict.csv')

LOCAL_SCHEMA_FILE = os.path.join(LOCAL_ROOT, 'schema.json')
LOCAL_FEATURES_FILE = os.path.join(LOCAL_ROOT, 'features.json')

if file_io.file_exists(LOCAL_ROOT):
  file_io.delete_recursively(LOCAL_ROOT)  
  
file_io.recursive_create_dir(LOCAL_ROOT)

The iris dataset is small, so the data is embedded into this notebook. Write the iris data set into 3 files: training, eval, prediction. Note that the prediction dataset does not have target values. 

In [36]:
%writefile {LOCAL_TRAIN_FILE}
Iris-setosa,4,4.6,3.1,1.5,0.2
Iris-setosa,20,5.1,3.8,1.5,0.3
Iris-setosa,43,4.4,3.2,1.3,0.2
Iris-versicolor,88,6.3,2.3,4.4,1.3
Iris-versicolor,76,6.6,3,4.4,1.4
Iris-versicolor,63,6,2.2,4,1
Iris-setosa,47,5.1,3.8,1.6,0.2
Iris-virginica,146,6.7,3,5.2,2.3
Iris-versicolor,53,6.9,3.1,4.9,1.5
Iris-versicolor,71,5.9,3.2,4.8,1.8
Iris-virginica,144,6.8,3.2,5.9,2.3
Iris-virginica,124,6.3,2.7,4.9,1.8
Iris-virginica,122,5.6,2.8,4.9,2
Iris-setosa,17,5.4,3.9,1.3,0.4
Iris-setosa,7,4.6,3.4,1.4,0.3
Iris-versicolor,87,6.7,3.1,4.7,1.5
Iris-virginica,131,7.4,2.8,6.1,1.9
Iris-setosa,2,4.9,3,1.4,0.2
Iris-virginica,147,6.3,2.5,5,1.9
Iris-setosa,29,5.2,3.4,1.4,0.2
Iris-versicolor,91,5.5,2.6,4.4,1.2
Iris-virginica,110,7.2,3.6,6.1,2.5
Iris-virginica,121,6.9,3.2,5.7,2.3
Iris-setosa,45,5.1,3.8,1.9,0.4
Iris-setosa,10,4.9,3.1,1.5,0.1
Iris-setosa,36,5,3.2,1.2,0.2
Iris-virginica,112,6.4,2.7,5.3,1.9
Iris-setosa,46,4.8,3,1.4,0.3
Iris-virginica,132,7.9,3.8,6.4,2
Iris-versicolor,77,6.8,2.8,4.8,1.4
Iris-setosa,6,5.4,3.9,1.7,0.4
Iris-versicolor,90,5.5,2.5,4,1.3
Iris-virginica,137,6.3,3.4,5.6,2.4
Iris-setosa,31,4.8,3.1,1.6,0.2
Iris-virginica,120,6,2.2,5,1.5
Iris-virginica,138,6.4,3.1,5.5,1.8
Iris-setosa,24,5.1,3.3,1.7,0.5
Iris-versicolor,96,5.7,3,4.2,1.2
Iris-versicolor,68,5.8,2.7,4.1,1
Iris-virginica,150,5.9,3,5.1,1.8
Iris-setosa,26,5,3,1.6,0.2
Iris-versicolor,98,6.2,2.9,4.3,1.3
Iris-versicolor,80,5.7,2.6,3.5,1
Iris-versicolor,72,6.1,2.8,4,1.3
Iris-versicolor,75,6.4,2.9,4.3,1.3
Iris-setosa,38,4.9,3.1,1.5,0.1
Iris-setosa,35,4.9,3.1,1.5,0.1
Iris-versicolor,89,5.6,3,4.1,1.3
Iris-versicolor,84,6,2.7,5.1,1.6
Iris-versicolor,51,7,3.2,4.7,1.4
Iris-virginica,116,6.4,3.2,5.3,2.3
Iris-versicolor,54,5.5,2.3,4,1.3
Iris-virginica,130,7.2,3,5.8,1.6
Iris-virginica,115,5.8,2.8,5.1,2.4
Iris-setosa,32,5.4,3.4,1.5,0.4
Iris-virginica,104,6.3,2.9,5.6,1.8
Iris-versicolor,64,6.1,2.9,4.7,1.4
Iris-setosa,18,5.1,3.5,1.4,0.3
Iris-versicolor,66,6.7,3.1,4.4,1.4
Iris-setosa,15,5.8,4,1.2,0.2
Iris-versicolor,52,6.4,3.2,4.5,1.5
Iris-virginica,103,7.1,3,5.9,2.1
Iris-setosa,9,4.4,2.9,1.4,0.2
Iris-versicolor,83,5.8,2.7,3.9,1.2
Iris-virginica,135,6.1,2.6,5.6,1.4
Iris-virginica,139,6,3,4.8,1.8
Iris-versicolor,85,5.4,3,4.5,1.5
Iris-virginica,106,7.6,3,6.6,2.1
Iris-setosa,27,5,3.4,1.6,0.4
Iris-virginica,140,6.9,3.1,5.4,2.1
Iris-versicolor,67,5.6,3,4.5,1.5
Iris-setosa,12,4.8,3.4,1.6,0.2
Iris-versicolor,56,5.7,2.8,4.5,1.3
Iris-virginica,113,6.8,3,5.5,2.1
Iris-versicolor,62,5.9,3,4.2,1.5
Iris-virginica,145,6.7,3.3,5.7,2.5
Iris-virginica,111,6.5,3.2,5.1,2
Iris-virginica,141,6.7,3.1,5.6,2.4
Iris-setosa,34,5.5,4.2,1.4,0.2
Iris-versicolor,81,5.5,2.4,3.8,1.1
Iris-setosa,8,5,3.4,1.5,0.2
Iris-virginica,129,6.4,2.8,5.6,2.1
Iris-versicolor,57,6.3,3.3,4.7,1.6
Iris-virginica,128,6.1,3,4.9,1.8
Iris-virginica,119,7.7,2.6,6.9,2.3
Iris-virginica,126,7.2,3.2,6,1.8
Iris-versicolor,58,4.9,2.4,3.3,1
Iris-virginica,117,6.5,3,5.5,1.8
Iris-virginica,127,6.2,2.8,4.8,1.8
Iris-setosa,16,5.7,4.4,1.5,0.4
Iris-setosa,3,4.7,3.2,1.3,0.2
Iris-virginica,108,7.3,2.9,6.3,1.8
Iris-virginica,118,7.7,3.8,6.7,2.2
Iris-setosa,42,4.5,2.3,1.3,0.3
Iris-virginica,142,6.9,3.1,5.1,2.3
Iris-setosa,14,4.3,3,1.1,0.1
Iris-virginica,134,6.3,2.8,5.1,1.5
Iris-versicolor,94,5,2.3,3.3,1
Iris-setosa,19,5.7,3.8,1.7,0.3
Iris-virginica,133,6.4,2.8,5.6,2.2
Iris-virginica,114,5.7,2.5,5,2
Iris-versicolor,86,6,3.4,4.5,1.6
Iris-versicolor,93,5.8,2.6,4,1.2
Iris-versicolor,92,6.1,3,4.6,1.4
Iris-virginica,109,6.7,2.5,5.8,1.8
Iris-virginica,102,5.8,2.7,5.1,1.9
Iris-setosa,41,5,3.5,1.3,0.3
Iris-versicolor,60,5.2,2.7,3.9,1.4
Iris-virginica,105,6.5,3,5.8,2.2
Iris-versicolor,65,5.6,2.9,3.6,1.3
Iris-setosa,28,5.2,3.5,1.5,0.2
Iris-versicolor,82,5.5,2.4,3.7,1
Iris-setosa,25,4.8,3.4,1.9,0.2
Iris-versicolor,79,6,2.9,4.5,1.5
Iris-setosa,1,5.1,3.5,1.4,0.2
Iris-versicolor,61,5,2,3.5,1
Iris-virginica,149,6.2,3.4,5.4,2.3
Iris-setosa,48,4.6,3.2,1.4,0.2
Iris-setosa,22,5.1,3.7,1.5,0.4
Iris-setosa,30,4.7,3.2,1.6,0.2

Writing ./iris_notebook_workspace/train.csv


In [37]:
%writefile {LOCAL_EVAL_FILE}
Iris-virginica,107,4.9,2.5,4.5,1.7
Iris-versicolor,100,5.7,2.8,4.1,1.3
Iris-versicolor,99,5.1,2.5,3,1.1
Iris-setosa,13,4.8,3,1.4,0.1
Iris-versicolor,70,5.6,2.5,3.9,1.1
Iris-setosa,11,5.4,3.7,1.5,0.2
Iris-setosa,37,5.5,3.5,1.3,0.2
Iris-versicolor,69,6.2,2.2,4.5,1.5
Iris-setosa,40,5.1,3.4,1.5,0.2
Iris-virginica,101,6.3,3.3,6,2.5
Iris-setosa,39,4.4,3,1.3,0.2
Iris-versicolor,74,6.1,2.8,4.7,1.2
Iris-versicolor,97,5.7,2.9,4.2,1.3
Iris-setosa,50,5,3.3,1.4,0.2
Iris-versicolor,95,5.6,2.7,4.2,1.3
Iris-setosa,44,5,3.5,1.6,0.6
Iris-virginica,123,7.7,2.8,6.7,2
Iris-setosa,23,4.6,3.6,1,0.2
Iris-versicolor,59,6.6,2.9,4.6,1.3
Iris-virginica,148,6.5,3,5.2,2
Iris-versicolor,55,6.5,2.8,4.6,1.5
Iris-setosa,49,5.3,3.7,1.5,0.2
Iris-versicolor,78,6.7,3,5,1.7
Iris-versicolor,73,6.3,2.5,4.9,1.5
Iris-virginica,136,7.7,3,6.1,2.3
Iris-setosa,33,5.2,4.1,1.5,0.1
Iris-virginica,125,6.7,3.3,5.7,2.1
Iris-virginica,143,5.8,2.7,5.1,1.9
Iris-setosa,21,5.4,3.4,1.7,0.2
Iris-setosa,5,5,3.6,1.4,0.2

Writing ./iris_notebook_workspace/eval.csv


In [38]:
%writefile {LOCAL_PREDICT_FILE}
107,4.9,2.5,4.5,1.7
100,5.7,2.8,4.1,1.3
99,5.1,2.5,3,1.1
13,4.8,3,1.4,0.1
70,5.6,2.5,3.9,1.1
11,5.4,3.7,1.5,0.2
37,5.5,3.5,1.3,0.2
69,6.2,2.2,4.5,1.5
40,5.1,3.4,1.5,0.2
101,6.3,3.3,6,2.5
39,4.4,3,1.3,0.2
74,6.1,2.8,4.7,1.2
97,5.7,2.9,4.2,1.3
50,5,3.3,1.4,0.2
95,5.6,2.7,4.2,1.3
44,5,3.5,1.6,0.6
123,7.7,2.8,6.7,2
23,4.6,3.6,1,0.2
59,6.6,2.9,4.6,1.3
148,6.5,3,5.2,2
55,6.5,2.8,4.6,1.5
49,5.3,3.7,1.5,0.2
78,6.7,3,5,1.7
73,6.3,2.5,4.9,1.5
136,7.7,3,6.1,2.3
33,5.2,4.1,1.5,0.1
125,6.7,3.3,5.7,2.1
143,5.8,2.7,5.1,1.9
21,5.4,3.4,1.7,0.2
5,5,3.6,1.4,0.2

Writing ./iris_notebook_workspace/predict.csv


<a name="local_preprocessing"></a>
Local preprocessing starting from csv files
=====

A schema file is used to describe each column of the csv files. It is assumed that the train, eval, and prediction csv files all have the same schema, but the prediction file has a missing target column. The format of the  schema file is a valid BigQuery table schema file. This allows BigQuery to be used later in cloud preprocessing. Only 3 BigQuery types are supported: STRING (for categorical columns) and INTEGER and FLOAT (for numerical columns).

In [39]:
schema = [
    {
        "name": "flower",
        "type": "STRING"
    },
    {
        "name": "key",
        "type": "INTEGER"
    },
    {
        "name": "sepal_length",
        "type": "FLOAT"
    },
    {
        "name": "sepal_width",
        "type": "FLOAT"
    },
    {
        "name": "petal_length",
        "type": "FLOAT"
    },
    {
        "name": "petal_width",
        "type": "FLOAT"
    }   
]

# Write schema to a file so that the cloud notebooks can use it.
file_io.write_string_to_file(
  LOCAL_SCHEMA_FILE,
  json.dumps(schema, indent=2))

In [40]:
!rm -fr {LOCAL_PREPROCESSING_DIR}

In [41]:
train_csv = ml.CsvDataSet(
  file_pattern=LOCAL_TRAIN_FILE,
  schema=schema)
eval_csv = ml.CsvDataSet(
  file_pattern=LOCAL_EVAL_FILE,
  schema=schema)

In [42]:
sd.local_preprocess(
  dataset=train_csv,
  output_dir=LOCAL_PREPROCESSING_DIR,
)

Starting local preprocessing.
Local preprocessing done.


The output of preprocessing is a numerical_analysis file that contains analysis from the numerical columns, and a vocab file from each categorical column. The files preoduced by preprocessing are consumed in training, and you should not have to worry about these files. Just for fun, lets look at them.

In [43]:
!ls  {LOCAL_PREPROCESSING_DIR}

numerical_analysis.json  schema.json  vocab_flower.csv


In [44]:
!cat {LOCAL_PREPROCESSING_DIR}/numerical_analysis.json

{
  "sepal_width": {
    "max": 4.4,
    "mean": 3.050833333333332,
    "min": 2.0
  },
  "petal_width": {
    "max": 2.5,
    "mean": 1.2324999999999995,
    "min": 0.1
  },
  "sepal_length": {
    "max": 7.9,
    "mean": 5.867500000000002,
    "min": 4.3
  },
  "key": {
    "max": 150.0,
    "mean": 76.73333333333333,
    "min": 1.0
  },
  "petal_length": {
    "max": 6.9,
    "mean": 3.830833333333335,
    "min": 1.1
  }
}

In [45]:
!cat {LOCAL_PREPROCESSING_DIR}/schema.json

[{"type": "STRING", "name": "flower"}, {"type": "INTEGER", "name": "key"}, {"type": "FLOAT", "name": "sepal_length"}, {"type": "FLOAT", "name": "sepal_width"}, {"type": "FLOAT", "name": "petal_length"}, {"type": "FLOAT", "name": "petal_width"}]

In [46]:
!cat {LOCAL_PREPROCESSING_DIR}/vocab_flower.csv

Iris-virginica
Iris-setosa
Iris-versicolor

<a name="local_training"></a>
Local Training
===========

The files in the output folder of preprocessing are consumed by the trainer. Training requires a transform config file to describe what transforms to apply on the data. The key and target transform are the only required transform, a default transform will be applied to every other column if it is not listed in the transforms.

In [47]:
features = {
  "sepal_length": {"transform": "scale"},
  "sepal_width": {"transform": "scale"},
  "key": {"transform": "key"},
  "flower": {"transform": "target"}
 }

# Write the features to a file so that the cloud notebooks can use the same features.
file_io.write_string_to_file(
  LOCAL_FEATURES_FILE,
  json.dumps(transforms, indent=2)
)

In [48]:
!rm -fr {LOCAL_TRAINING_DIR}

In [49]:
sd.local_train(
  train_dataset=train_csv,
  eval_dataset=eval_csv,
  transforms=features,
  preprocess_output_dir=LOCAL_PREPROCESSING_DIR,
  output_dir=LOCAL_TRAINING_DIR,
  model_type='linear_classification',
  top_n=3,
  max_steps=1000,
)

Starting local training.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87717d2f50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87717d2f50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: ./iris_notebook_workspace/training/train/export/intermediate_prediction_models/1488320547453/saved_model.pb


INFO:tensorflow:SavedModel written to: ./iris_notebook_workspace/training/train/export/intermediate_prediction_models/1488320547453/saved_model.pb


Local training done.


In [64]:
!ls {LOCAL_TRAINING_DIR}

evaluation_model  model  train	transforms_file.json


<a name="local_prediction"></a>
Local prediction
================

Local predict uses the model produced by training. The input data can be a csv string or Pandas DataFrame, but the schema must match the data set used for training, except the target column is missing. That is, if the training dataset had the values "id,target,value1,value2", the prediction data must be in the form "id,value1,value2".

In [51]:
sd.local_predict(
  training_ouput_dir=LOCAL_TRAINING_DIR,
  data=['101,6.3,3.3,6,2.5',
        '107,4.9,2.5,4.5,1.7',
        '100,5.7,2.8,4.1,1.3']
)

Starting local prediction.
Local prediction done.


,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.940259,Iris-versicolor,0.059701,Iris-setosa,0.000040
1,107,Iris-versicolor,0.753490,Iris-virginica,0.232542,Iris-setosa,0.013968
2,100,Iris-versicolor,0.840116,Iris-virginica,0.145837,Iris-setosa,0.014048


In [52]:
import pandas as pd
sd.local_predict(
  training_ouput_dir=LOCAL_TRAINING_DIR,
  data=pd.DataFrame(
    [[101,6.3,3.3,6,2.5],
     [107,4.9,2.5,4.5,1.7],
     [100,5.7,2.8,4.1,1.3]])
)

Starting local prediction.
Local prediction done.


,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.940259,Iris-versicolor,0.059701,Iris-setosa,0.000040
1,107,Iris-versicolor,0.753490,Iris-virginica,0.232542,Iris-setosa,0.013968
2,100,Iris-versicolor,0.840116,Iris-virginica,0.145837,Iris-setosa,0.014048


<a name="local_batch_prediction"></a>
Local batch prediction
============

Local batch prediction runs prediction on batched input data. This is ideal if the input dataset is very large or you have limited available main memory. However, for very large datasets, it is better to run batch prediction using the Google Cloud Machine Learning Engine services. Two output formats are supported, csv and json. The output may also be shardded. Another feature of batch prediction is the option to run evaluation--prediction on data that contains the target column. Like local_predict, the input data must batch the schema used for training.

In [53]:
!rm -fr {LOCAL_BATCH_PREDICTION_DIR}

In [54]:
sd.local_batch_predict(
  training_ouput_dir=LOCAL_TRAINING_DIR,
  prediction_input_file=LOCAL_EVAL_FILE,
  output_dir=LOCAL_BATCH_PREDICTION_DIR,
  output_format='csv',
  mode='evaluation'
)

Starting local batch prediction.
Local batch prediction done.


In [55]:
!ls {LOCAL_BATCH_PREDICTION_DIR}

csv_schema.json  errors-00000-of-00001.txt  predictions-00000-of-00001.csv


In [56]:
!cat  {LOCAL_BATCH_PREDICTION_DIR}/csv*

[
  {
    "type": "INTEGER", 
    "mode": "NULLABLE", 
    "name": "key"
  }, 
  {
    "type": "STRING", 
    "mode": "NULLABLE", 
    "name": "target_from_input"
  }, 
  {
    "type": "STRING", 
    "mode": "NULLABLE", 
    "name": "top_1_label"
  }, 
  {
    "type": "FLOAT", 
    "mode": "NULLABLE", 
    "name": "top_1_score"
  }, 
  {
    "type": "STRING", 
    "mode": "NULLABLE", 
    "name": "top_2_label"
  }, 
  {
    "type": "FLOAT", 
    "mode": "NULLABLE", 
    "name": "top_2_score"
  }, 
  {
    "type": "STRING", 
    "mode": "NULLABLE", 
    "name": "top_3_label"
  }, 
  {
    "type": "FLOAT", 
    "mode": "NULLABLE", 
    "name": "top_3_score"
  }
]


In [57]:
!cat {LOCAL_BATCH_PREDICTION_DIR}/errors*

In [58]:
!head {LOCAL_BATCH_PREDICTION_DIR}/predictions-00000*

107,Iris-virginica,Iris-versicolor,0.753490149975,Iris-virginica,0.23254224658,Iris-setosa,0.0139675997198
100,Iris-versicolor,Iris-versicolor,0.840115904808,Iris-virginica,0.145836502314,Iris-setosa,0.01404756587
99,Iris-versicolor,Iris-versicolor,0.842605233192,Iris-setosa,0.105508580804,Iris-virginica,0.0518861338496
13,Iris-setosa,Iris-setosa,0.952213346958,Iris-versicolor,0.0477338135242,Iris-virginica,5.28481396032e-05
70,Iris-versicolor,Iris-versicolor,0.907238066196,Iris-virginica,0.0812551751733,Iris-setosa,0.0115067921579
11,Iris-setosa,Iris-setosa,0.971967577934,Iris-versicolor,0.0279693827033,Iris-virginica,6.30639042356e-05
37,Iris-setosa,Iris-setosa,0.950098335743,Iris-versicolor,0.049751047045,Iris-virginica,0.000150677951751
69,Iris-versicolor,Iris-versicolor,0.533427774906,Iris-virginica,0.466347098351,Iris-setosa,0.00022518968035
40,Iris-setosa,Iris-setosa,0.965074837208,Iris-versicolor,0.0348613634706,Iris-virginica,6.38756901026e-05
101,Iris-virginica,Iris-virginica

In [59]:
!rm -fr {LOCAL_BATCH_PREDICTION_DIR}

In [60]:
sd.local_batch_predict(
  training_ouput_dir=LOCAL_TRAINING_DIR,
  prediction_input_file=LOCAL_PREDICT_FILE,
  output_dir=LOCAL_BATCH_PREDICTION_DIR,
  output_format='json',
  mode='prediction'
)

Starting local batch prediction.
Local batch prediction done.


In [61]:
!ls {LOCAL_BATCH_PREDICTION_DIR}

errors-00000-of-00001.txt  predictions-00000-of-00001.json


In [62]:
!head {LOCAL_BATCH_PREDICTION_DIR}/predictions*

{"top_2_label": "Iris-virginica","top_3_score": 0.013967599719762802,"top_1_label": "Iris-versicolor","top_2_score": 0.2325422465801239,"top_3_label": "Iris-setosa","top_1_score": 0.753490149974823,"key": 107}
{"top_2_label": "Iris-virginica","top_3_score": 0.014047565869987011,"top_1_label": "Iris-versicolor","top_2_score": 0.1458365023136139,"top_3_label": "Iris-setosa","top_1_score": 0.8401159048080444,"key": 100}
{"top_2_label": "Iris-setosa","top_3_score": 0.05188613384962082,"top_1_label": "Iris-versicolor","top_2_score": 0.10550858080387115,"top_3_label": "Iris-virginica","top_1_score": 0.8426052331924438,"key": 99}
{"top_2_label": "Iris-versicolor","top_3_score": 5.2848139603156596e-05,"top_1_label": "Iris-setosa","top_2_score": 0.047733813524246216,"top_3_label": "Iris-virginica","top_1_score": 0.9522133469581604,"key": 13}
{"top_2_label": "Iris-virginica","top_3_score": 0.011506792157888412,"top_1_label": "Iris-versicolor","top_2_score": 0.08125517517328262,"top_3_label": "Ir

Clean up
===

As everything was written to LOCAL_ROOT, we can simply remove this folder. If you want to delete those files, uncomment and run the next cell.

In [63]:
#!rm -fr {LOCAL_ROOT}